# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read csv as dataframe
file = "../weather_data.csv"

weather_df = pd.read_csv(file)
weather_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,busselton,AU,-33.65,115.33,66.99,60,10,1.01
1,bluff,NZ,-46.60,168.33,55.99,79,37,4.00
2,chegdomyn,RU,51.12,133.02,61.59,56,98,1.79
3,otradnoye,RU,59.78,30.82,50.00,99,27,4.92
4,mogwase,ZA,-25.18,27.27,51.10,78,0,3.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
# Store coordinates into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Store humidity as float
humidity = weather_df["Humidity (%)"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=max(weather_df["Humidity (%)"]),
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = weather_df.loc[(weather_df["Max Temperature (F)"] < 80) &
                          (weather_df["Max Temperature (F)"] > 70) &
                          (weather_df["Wind Speed (mph)"] < 10) & 
                          (weather_df["Cloudiness (%)"] == 0)]

ideal_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
14,inverell,AU,-29.78,151.12,72.00,33,0,8.99
91,buraydah,SA,26.33,43.98,73.40,35,0,4.70
102,iranshahr,IR,27.20,60.68,75.20,19,0,4.70
171,riyadh,SA,24.69,46.72,77.00,20,0,5.82
267,garmsar,IR,35.22,52.34,73.04,31,0,4.85
277,zhangye,CN,38.93,100.45,70.20,24,0,6.24
375,yumen,CN,40.28,97.20,70.54,22,0,8.50
388,rawah,IQ,34.48,41.91,77.52,40,0,9.82
408,khorramshahr,IR,30.44,48.17,75.20,47,0,4.70
449,erdemli,TR,36.60,34.31,76.86,70,0,7.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Run a test to confirm approach for obtaining the data

target_coordinates = "-29.78, 151.12"

params = {
        "location": target_coordinates,
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))
print(response["results"][0]["name"])

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.775878,
                    "lng": 151.102638
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.77439322010728,
                        "lng": 151.1040565298927
                    },
                    "southwest": {
                        "lat": -29.77709287989272,
                        "lng": 151.1013568701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Inverell Terrace Motor Lodge",
            "photos": [
                {
                    "height": 1960,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/116730577060529538079\">A Google 

In [8]:
# Create hotel dataframe
hotel_df = ideal_df

hotel_list = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop through each row in the dataframe 
for index, row in hotel_df.iterrows():
    
    # Set coordinates of city
    lat = row["Latitude"]
    lon = row["Longitude"]
    target_coordinates = f"{lat},{lon}"

    # Set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key}
    
    # Run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    try:
        hotel_list.append(response["results"][0]["name"])
    except:
        hotel_list.append("")
        
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

C:\Users\mcogg\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
14,inverell,AU,-29.78,151.12,72.00,33,0,8.99,Inverell Terrace Motor Lodge
91,buraydah,SA,26.33,43.98,73.40,35,0,4.70,Mövenpick Hotel Qassim
102,iranshahr,IR,27.20,60.68,75.20,19,0,4.70,هتل و رستوران قصر
171,riyadh,SA,24.69,46.72,77.00,20,0,5.82,Hyatt Regency Riyadh Olaya
267,garmsar,IR,35.22,52.34,73.04,31,0,4.85,هتل شهر آفتاب Sun City


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color = 'rgba(0, 150, 0, 0.4)',
    stroke_color = 'rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content = hotel_info)

# Display combined map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))